## Excel Automation with Python

#### Agenda 

- Automating Excel with openpyxl
- Automating Excel with xlwings
- Converting VBA Code to Python Scripts
- Creating Python-based Excel Plugins/Add-ins
- Creating a macro-enabled Excel file using xlwings
- Creating Custom Excel Plugins to Execute Python Scripts

#### Comparison between `openpyxl` and `xlwings`

|    Feature                         | `openpyxl`                                   | `xlwings`                                        |
| ---------------------------------- | -------------------------------------------- | ------------------------------------------------ |
| **1. Excel Installation Required** |  No (pure Python)                            |  Yes (requires Excel installed)                  |
| **2. Real-time Excel Interaction** |  File-based only                             |  Controls open Excel app (like VBA)              |
| **3. Macro & UDF Support**         |  Cannot run macros or define UDFs            |  Can run macros and define Python-based UDFs     |
| **4. Performance**                 |  Faster for large-scale, headless automation |  Slower due to Excel COM interface               |
| **5. Charting & Visual Updates**   |  Supports static chart creation              |  Live chart creation, UI updates, event handling |

* Use **`openpyxl`** for lightweight, cross-platform, headless tasks like report generation or formatting.
* Use **`xlwings`** when you need **live Excel control**, **macros**, or **tight Excel-Python integration** like VBA replacements.

### Automating Excel with openpyxl

#### What is openpyxl?
openpyxl is a powerful Python library that allows you to read, write, and modify Excel files (.xlsx, not .xls) without needing Microsoft Excel installed.

**It's especially useful for:**

- Automating reports
- Modifying templates
- Reading structured data
- Applying formatting, formulas, charts, etc.

In [ ]:
pip install openpyxl

In [2]:
import os 
os.chdir(r"C:\Users\vaide\OneDrive - knowledgecorner.in\Course Material\Clients\Virtua Search\Excel Python Integration\Files")

###### Ex. Create a New Workbook

In [ ]:
from openpyxl import Workbook

wb = Workbook()
ws = wb.active  # Get the default sheet
ws.title = "MySheet"

ws['A1'] = "Hello"
ws['B1'] = 123

wb.save("myfile.xlsx")

###### Ex. Load Existing Workbook

In [ ]:
from openpyxl import load_workbook

wb = load_workbook("myfile.xlsx")
ws = wb.active  # Or wb['SheetName']

###### Ex. Read/Write Cell Values

In [ ]:
# Reading
value = ws['A1'].value
print(value)

# Writing
ws['A2'] = "New Value"


###### Ex. Loop Through Rows/Columns

In [ ]:
# Iterate through column and rows
for column in ws.columns:
    for row in column:
        print(row.value)

In [ ]:
# Iterate through rows and columns
for row in ws.rows:
    for col in row:
        print(col.value)

###### Ex. Add New Sheet or Access Existing Sheet

In [ ]:
# Create new sheet
ws2 = wb.create_sheet(title="Summary")

# Access existing
ws_existing = wb["MySheet"]

wb.save("myfile.xlsx")

In [ ]:
from openpyxl import load_workbook
from openpyxl.worksheet.datavalidation import DataValidation
from openpyxl.utils import get_column_letter
from openpyxl.styles import Protection

wb = load_workbook("input.xlsx")
ws = wb.active

new_cols = ["Rating", "Bonus", "Tax", "Net Salary"]
existing_headers = [cell.value for cell in ws[1]]

# Add new columns only if not present
for col_name in new_cols:
    if col_name not in existing_headers:
        ws.cell(row=1, column=ws.max_column + 1, value=col_name)

# Refresh header map and prepare col_letter lookup
headers = [cell.value for cell in ws[1]]
col_letters = {name: get_column_letter(idx + 1) for idx, name in enumerate(headers)}

# Apply data validation for "Rating"
rating_col = col_letters["Rating"]
rating_range = f"{rating_col}2:{rating_col}{ws.max_row}"
dv = DataValidation(type="list", formula1='"Excellent,Good,Average,Below Average,Poor"', allow_blank=False)
dv.prompt = "Choose a performance rating"
dv.error = "Select from the list"
ws.add_data_validation(dv)
dv.add(rating_range)

# Add formulas row-wise
for row in range(2, ws.max_row + 1):
    salary = f"{col_letters['Salary']}{row}"
    rating = f"{col_letters['Rating']}{row}"
    bonus = f"{col_letters['Bonus']}{row}"
    tax = f"{col_letters['Tax']}{row}"
    net = f"{col_letters['Net Salary']}{row}"

    ws[bonus] =  f'=IF({rating}="Excellent", {salary} * 0.12, IF({rating}="Good", {salary} * 0.1, IF({rating}="Average", {salary} * 0.07, IF({rating}="Below Average", {salary} * 0.05, {salary} * 0.01))))'
    ws[tax] = f"=({salary}+{bonus})*0.1"
    ws[net] = f"={salary}+{bonus}-{tax}"

Hide intermediate columns
for col in ["Bonus", "Tax"]:
    ws.column_dimensions[col_letters[col]].hidden = True

# Lock all cells except Rating column (hide formulas)
rating_col_idx = headers.index("Rating") + 1
for row in ws.iter_rows(min_row=2):
    for cell in row:
        if cell.column == rating_col_idx:
            cell.protection = Protection(locked=False, hidden=True)
        else:
            cell.protection = Protection(locked=True, hidden=True)

# Protect worksheet
ws.protection.enable()
ws.protection.set_password("yourpassword")

wb.save("input.xlsx")


### Automating Excel with xlwings

#### What is xlwings?

xlwings is a Python library that enables automated interaction with Excel using the Excel COM API. Unlike openpyxl, it requires Microsoft Excel installed and allows live interaction with open workbooks, making it great for:

- Excel automation
- User-defined functions (UDFs)
- Integration with Excel macros
- Interactive dashboards or calculators

#### Quick tools - 

Here's a handy **`xlwings` Cheatsheet** for quick reference — covering the **most commonly used operations** with examples.

**Open or Create Workbook**

```python
import xlwings as xw

wb = xw.Book("file.xlsx")      # Open existing
wb = xw.Book()                 # New workbook
wb.save("file.xlsx")          # Save as
wb.close()                    # Close workbook
```
---
**Access Sheets**

```python
ws = wb.sheets[0]             # By index
ws = wb.sheets['Sheet1']      # By name
ws = wb.sheets.add("New")     # Add new sheet
ws.delete()                   # Delete sheet
```

---

**Read/Write Data**

```python
ws.range("A1").value = "Hello"             # Write single value
value = ws.range("A1").value               # Read value

ws.range("A1:C3").value = [[1,2,3],[4,5,6]]  # Write 2D list
data = ws.range("A1").expand().value        # Read table
```

---

**Charts**

```python
chart = ws.charts.add()
chart.chart_type = 'line'
chart.set_source_data(ws.range("A1:B10"))
chart.api[1].ChartTitle.Text = "Sales Trend"
```

---

**Formatting**

```python
cell = ws.range("A1")
cell.color = (255, 255, 0)              # Background color
cell.api.Font.Bold = True              # Bold
cell.api.Font.Color = 0xFF0000         # Red font
cell.api.NumberFormat = "$#,##0.00"    # Currency format
```

---

**Looping Ranges**

```python
for cell in ws.range("A1:A5"):
    print(cell.value)
```

---

**Data Validation**

```python
ws.range("A1").api.Validation.Add(
    Type=3,  # List
    Formula1="Option1,Option2,Option3"
)
```

---

**Sheet & Cell Protection**

```python
ws.api.Protect(Password="mypassword")

cell = ws.range("A2")
cell.api.Locked = False
cell.api.FormulaHidden = True
```

---

**PasteSpecial (formats only)**

```python
ws.range("A1:D1").api.Copy()
ws.range("A2:D2").api.PasteSpecial(Paste=-4122)  # Formats only
```

---

**Used Range, Last Row/Col**

```python
ws.used_range.rows.count
ws.used_range.columns.count

last_row = ws.range("A1").end("down").row
last_col = ws.range("A1").end("right").column
```

---

**Tips**

* `xw.Book().app.visible = False` to run Excel in background
* `.formula` vs `.value`: to write Excel formulas like `=SUM(A1:A5)`
* Always `save()` before closing to persist changes

---

In [ ]:
pip install xlwings

###### Ex. Open or Connect to Excel Workbook

In [ ]:
import xlwings as xw

# Start a new instance of Excel and open a workbook
wb = xw.Book("input.xlsx")  # Opens existing file
# wb = xw.Book()  # Opens a new workbook

###### Ex. Access Sheets and Cells

In [ ]:
sheet = wb.sheets["Sheet1"]

# Read and write values
# sheet["A1"].value = "Hello, Excel"
# print(sheet["A1"].value)

###### Ex. Count number of rows and columns

In [ ]:
ws = xw.Book("input.xlsx").sheets[0]
rows = ws.used_range.rows.count
cols = ws.used_range.columns.count
rows, cols

In [ ]:
for i in ws.used_range.rows :
    print(i.value)

In [ ]:
for i in ws.used_range.columns :
    print(i.value)

###### Ex. Read/Write Ranges

In [ ]:
# Writing a list to Excel
sheet.range("A2:A5").value = ["One", "Two", "Three", "Four"]

# Reading back a range
data = sheet.range("A2:A5").value
print(data)

###### Ex. Loop and Update Multiple Cells

In [ ]:
for i in range(1, 6):
    sheet.range(f"B{i}").value = i * 10

###### Ex. Save and Close Workbook

In [ ]:
wb.save("updated_example.xlsx")
wb.close()